In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from IPython.display import clear_output
from tkinter import Tk, filedialog
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from quick_pp.objects import Project
import quick_pp.las_handler as las
from quick_pp.core_calibration import fit_poroperm_curve, poroperm_xplot
from quick_pp.rock_type import rock_typing, rqi, fzi, vsh_gr
from quick_pp.plotter import plotly_log

## Load and merge well data and core data

In [ ]:
# Load well from saved file
project = "M3"
project_path = rf"data\04_project\{project}.qppp"
project = Project().load(project_path)
project.get_well_names()

In [ ]:
col_rename = {
    'NEUT': 'NPHI',
    'NEUTHC': 'NPHI_HC',
    'DENB': 'RHOB',
    'DENBHC': 'RHOB_HC',
}

In [ ]:
# Load data
well_name = 'M03-2'  # 'M01-3'
well = project.get_well(well_name)
well_data = well.data
well_data = well_data.rename(columns=col_rename)

min_depth = well_data.DEPTH.min()
max_depth = well_data.DEPTH.max()
well_data = well_data[(well_data.DEPTH >= min_depth) & (well_data.DEPTH <= max_depth)].copy()

In [ ]:
core_df = pd.read_csv(r"C:\Users\imran\OneDrive\Desktop\Petrophysics\Data\Core\M1_M3_RAW_SCAL.csv")
core_df = core_df[core_df.WELL_NAME == well_name].sort_values('DEPTH')

In [ ]:
well_data = pd.merge_asof(well_data, core_df[['DEPTH', 'CPORE', 'CPERM']], on='DEPTH',
                          direction='nearest', tolerance=1)

## Rock Typing

In [ ]:
a, b = fit_poroperm_curve(core_df['CPORE'], core_df['CPERM'])
for i, data in core_df.groupby('SAMPLE_NO'):
    poroperm_xplot(data['CPORE'], data['CPERM'], a, b, label=f'RT {i} - a: {a}, b: {b}')

In [ ]:
core_df['RQI'] = rqi(core_df['CPERM'], core_df['CPORE'])
core_df['FZI'] = fzi(core_df['CPERM'], core_df['CPORE'])

for rt, data in core_df.groupby('FZI'):
    a, b = fit_poroperm_curve(data['CPORE'], data['CPERM'])
    poroperm_xplot(data['CPORE'], data['CPERM'], a, b, log_log=True,
                   label=f'RT {rt} - a: {a}, b: {b}')


In [ ]:
poroperm_constants = {
    17: {'a': 1.5e3, 'b': 2.8},
    48: {'a': 3.5e3, 'b': 2.8},
    139: {'a': 3.3e1, 'b': 2.8},
    166: {'a': 9e2, 'b': 2.8},
    238: {'a': 7.5e2, 'b': 2.8},
    240: {'a': 1.5e1, 'b': 2.8},
    245: {'a': 5.5e1, 'b': 2.8}
}
for i, rt_data in core_df.groupby('SAMPLE_NO'):
    a = poroperm_constants[i]['a']
    b = poroperm_constants[i]['b']    
    poroperm_xplot(rt_data['CPORE'], rt_data['CPERM'], a, b, log_log=True,
                   label=f'SAMPLE #{i} - a: {a}, b: {b}')

In [ ]:
well_data['FZI'] = fzi(well_data['PERM'], well_data['PHIT'])
well_data['ROCK_FLAG'] = rock_typing(well_data['FZI'], [.5, 1.3, 1.7], higher_is_better=True)
fig = plotly_log(well_data)
fig.show(config=dict(scrollZoom=True))
# fig.write_html(rf"{well_name}_log.html", config=dict(scrollZoom=True))

In [ ]:
core_df = pd.merge_asof(core_df, well_data[['DEPTH', 'ROCK_FLAG']], on='DEPTH', direction='nearest')

## PORO-PERM

In [ ]:
poroperm_constants = {}
for rt, rt_data in core_df.groupby('ROCK_FLAG'):
    a, b = fit_poroperm_curve(rt_data['CPORE'], rt_data['CPERM'])
    poroperm_constants[rt] = {'a': a, 'b': b}
    poroperm_xplot(rt_data['CPORE'], rt_data['CPERM'], a, b, label=f'RT {rt} - a: {a}, b: {b}')
    # plt.show()


In [ ]:
poroperm_constants

In [ ]:
from quick_pp.core_calibration import fit_poroperm_curve, poroperm_xplot

poroperm_constants = {
    1.0: {'a': 624556, 'b': 4.325},
    2.0: {'a': 100029, 'b': 3.832},
    3.0: {'a': 131015, 'b': 4.666},
    4.0: {'a': 26971, 'b': 4.308}
}
for rt, rt_data in df.groupby('ROCK_FLAG'):
    a = poroperm_constants[rt]['a']
    b = poroperm_constants[rt]['b']    
    poroperm_xplot(rt_data['CPORE'], rt_data['CPERM'], a, b, label=f'RT {rt} - a: {a}, b: {b}')
    # plt.show()


In [ ]:
from quick_pp.core_calibration import fit_j_curve, j_xplot

well_name = 'BA-H010'
well_df = df[df['WELL_NAME'] == well_name]
well_df.dropna(axis=1, how='all', inplace=True)

j_constants = {}
for rt, well_data in well_df.groupby('RT_V2_2023'):
    temp_df = pd.DataFrame()
    for i, data in well_data.iterrows():
        t_df = data[[c for c in well_data.columns if 'SW[' in c]].reset_index(drop=True).T.reset_index(name='SW')
        # t_df = pd.concat([t_df, data[[c for c in well_data.columns if 'PC' in c]].reset_index(drop=True).T.reset_index(name='PC')], axis=1)
        # t_df = pd.concat([t_df, data[[c for c in well_data.columns if 'SWN' in c]].reset_index(drop=True).T.reset_index(name='SWN')], axis=1)
        t_df = pd.concat([t_df, data[[c for c in well_data.columns if 'J' in c]].reset_index(drop=True).T.reset_index(name='J')], axis=1)
        t_df.drop(columns='index', inplace=True)
        temp_df = pd.concat([temp_df, t_df])
    temp_df.dropna(inplace=True)
    temp_df = temp_df[(temp_df > 0) & (temp_df != np.inf)]
    temp_df = temp_df.sort_values(by='SW').reset_index(drop=True)
    a, b = fit_j_curve(temp_df['SW'], temp_df['J'])
    j_constants[rt] = {'a': a, 'b': b}
    j_xplot(temp_df['SW'], temp_df['J'], a, b, label=f"RT {rt} - a: {a}, b: {b}")
    # plt.show()


In [ ]:
j_constants

In [ ]:
from quick_pp.core_calibration import fit_j_curve, j_xplot

well_name = 'BA-H010'
well_df = df[df['WELL_NAME'] == well_name]
well_df.dropna(axis=1, how='all', inplace=True)

j_constants = {
    # 3.0: {'a': 0.507, 'b': -2.445}, 
    # 4.0: {'a': 0.583, 'b': -1.969}
    3.0: {'a': 0.0271, 'b': -2.653}, 
    4.0: {'a': 0.0363, 'b': -3.483}
}
for rt, well_data in well_df.groupby('RT_V2_2023'):
    temp_df = pd.DataFrame()
    for i, data in well_data.iterrows():
        t_df = data[[c for c in well_data.columns if 'SW[' in c]].reset_index(drop=True).T.reset_index(name='SW')
        # t_df = pd.concat([t_df, data[[c for c in well_data.columns if 'PC' in c]].reset_index(drop=True).T.reset_index(name='PC')], axis=1)
        # t_df = pd.concat([t_df, data[[c for c in well_data.columns if 'SWN' in c]].reset_index(drop=True).T.reset_index(name='SWN')], axis=1)
        t_df = pd.concat([t_df, data[[c for c in well_data.columns if 'J' in c]].reset_index(drop=True).T.reset_index(name='J')], axis=1)
        t_df.drop(columns='index', inplace=True)
        temp_df = pd.concat([temp_df, t_df])
    temp_df.dropna(inplace=True)
    temp_df = temp_df[(temp_df > 0) & (temp_df != np.inf)]
    temp_df = temp_df.sort_values(by='SW').reset_index(drop=True)
    a = j_constants[rt]['a']
    b = j_constants[rt]['b']
    j_xplot(temp_df['SW'], temp_df['J'], a, b, label=f"RT {rt} - a: {a}, b: {b}")
    # plt.show()


In [ ]:
# Merge core data with well data
well_df_core = well_df.dropna(subset=['CPORE', 'CPERM']).sort_values(by='DEPTH')
filtered_df = pd.merge_asof(barm_h010, well_df_core[['DEPTH', 'CPERM', 'CPORE']], on='DEPTH', direction='nearest', tolerance=0.5)

In [ ]:
# filtered_df.to_csv('barm_h010_core.csv', index=False)
filtered_df = pd.read_csv('barm_h010_core.csv')

In [ ]:
min_depth = 4650
max_depth = 4745
test = filtered_df[(filtered_df.DEPTH > min_depth) & (filtered_df.DEPTH < max_depth)][['DEPTH', 'PHIT', 'CPORE']]
plt.figure(figsize=(20, 3))
plt.plot(test.DEPTH, test.PHIT, label='PHIT')
plt.scatter(x=test.DEPTH, y=test.CPORE, label='CPORE')

In [ ]:
min_depth = 4650
max_depth = 4745
test = filtered_df[(filtered_df.DEPTH > min_depth) & (filtered_df.DEPTH < max_depth)][['DEPTH', 'PERM', 'CPERM']]
plt.figure(figsize=(20, 3))
plt.plot(test.DEPTH, test.PERM, label='PERM')
plt.scatter(x=test.DEPTH, y=test.CPERM, label='CPERM')
plt.yscale('log')

In [ ]:
from quick_pp.core_calibration import sw_shf_leverett_j
min_depth = 4600
max_depth = 4745
test = filtered_df[(filtered_df.DEPTH > min_depth) & (filtered_df.DEPTH < max_depth)].copy()
# TODO: groupby RT and use
test['SW_SHF'] = sw_shf_leverett_j(
    test['PERM'], test['PHIT'], test['DEPTH'], 
    fwl=4680,
    ift=26,
    gw=0.433,
    ghc=0.35,
    a=.038271,
    b=-5.33
)
plt.figure(figsize=(20, 3))
plt.plot(test.DEPTH, test.SWT, label='SWT')
plt.plot(test.DEPTH, test.SW_SHF, label='SW_SHF')
plt.legend()

In [ ]:
from quick_pp.core_calibration import fit_j_curve, bvw_xplot

well_name = 'BA-H010'
well_df = df[df['WELL_NAME'] == well_name]
well_df.dropna(axis=1, how='all', inplace=True)

for rt, well_data in well_df.groupby('RT_V2_2023'):
    temp_df = pd.DataFrame()
    for i, data in well_data.iterrows():
        cpore = data['CPORE']
        t_df = data[[c for c in well_data.columns if 'SW[' in c]].reset_index(drop=True).T.reset_index(name='SW')
        t_df['BVW'] = t_df['SW'] * cpore
        t_df = pd.concat([t_df, data[[c for c in well_data.columns if 'PC' in c]].reset_index(drop=True).T.reset_index(name='PC')], axis=1)
        # t_df = pd.concat([t_df, data[[c for c in well_data.columns if 'SWN' in c]].reset_index(drop=True).T.reset_index(name='SWN')], axis=1)
        # t_df = pd.concat([t_df, data[[c for c in well_data.columns if 'J' in c]].reset_index(drop=True).T.reset_index(name='J')], axis=1)
        t_df.drop(columns='index', inplace=True)
        temp_df = pd.concat([temp_df, t_df])
    temp_df.dropna(inplace=True)
    temp_df = temp_df[(temp_df > 0) & (temp_df != np.inf)]
    temp_df = temp_df.sort_values(by='SW').reset_index(drop=True)
    a, b = fit_j_curve(temp_df['BVW'], temp_df['PC'])
    bvw_xplot(temp_df['BVW'], temp_df['PC'], a, b, label=f"RT {rt} - a: {a}, b: {b}")
    # plt.show()


In [ ]:
from quick_pp.core_calibration import sw_shf_cuddy
min_depth = 4600
max_depth = 4745
test = filtered_df[(filtered_df.DEPTH > min_depth) & (filtered_df.DEPTH < max_depth)].copy()

test['SHF'] = sw_shf_cuddy(test['PHIT'], test['DEPTH'],
                            fwl=4750, 
                            gw=0.433, ghc=0.35, a=0.27, b=-2.0124)

plt.figure(figsize=(20, 3))
plt.plot(test.DEPTH, test.SWT, label='SWT')
plt.plot(test.DEPTH, test.SHF, label='SHF')
plt.legend()

In [ ]:
from quick_pp.core_calibration import sw_shf_cuddy
filtered_df['SHF'] = sw_shf_cuddy(filtered_df['PHIT'], filtered_df['DEPTH'], fwl=4750, gw=0.433, ghc=0.35, a=0.27, b=-2.0124)

In [ ]:
from quick_pp.core_calibration import sw_shf_choo
min_depth = 4600
max_depth = 4745
test = filtered_df[(filtered_df.DEPTH > min_depth) & (filtered_df.DEPTH < max_depth)].copy()

test['SHF'] = sw_shf_choo(test['PERM'], test['PHIT'], test['PHIE'], test['DEPTH'],
                            fwl=4680, ift=26,
                            gw=0.433, ghc=0.35, b0=0.4)

plt.figure(figsize=(20, 3))
plt.plot(test.DEPTH, test.SWT, label='SWT')
plt.plot(test.DEPTH, test.SHF, label='SHF')
plt.legend()

In [ ]:
from quick_pp.core_calibration import sw_shf_choo
filtered_df['SHF'] = sw_shf_choo(filtered_df['PERM'], filtered_df['PHIT'], filtered_df['PHIE'], filtered_df['DEPTH'], fwl=4680, ift=26, gw=0.433, ghc=0.35, b0=0.4)

In [ ]:
from quick_pp.core_calibration import sw_shf_leverett_j

j_constants = {
    1: {'a': 0.0271, 'b': -2.653},
    2: {'a': 0.0363, 'b': -3.483},
    3: {'a': 0.0382, 'b': -5.330}, 
    4: {'a': 0.0563, 'b': -8.483}
}
for rt, well_data in filtered_df.groupby('ROCK_FLAG'):
    a = j_constants[rt]['a']
    b = j_constants[rt]['b']
    filtered_df.loc[filtered_df['ROCK_FLAG'] == rt, 'SHF'] = sw_shf_leverett_j(
        well_data['PERM'], well_data['PHIT'], well_data['DEPTH'], 
        fwl=4680,
        ift=26,
        gw=0.433,
        ghc=0.35,
        a=a,
        b=b
    )

In [ ]:
from quick_pp.plotter import plotly_log

fig = plotly_log(filtered_df)
fig.show(config=dict(scrollZoom=True))
# fig.write_html(rf"{well_name}_log.html", config=dict(scrollZoom=True))

In [ ]:
from quick_pp.core_calibration import estimate_hafwl

hafwl = estimate_hafwl(
    sw=well_df['SW[0]'],
    poro=well_df['CPORE'],
    perm=well_df['CPERM'],
    ift=26,
    gw=1.0,
    ghc=0.7,
    a=0.507,
    b=2.455
)
hafwl

In [ ]:
# # Need HAFWL to plot 
# df.plot(x='BVW', y='DEPTH_SHIFTED', kind='scatter', logy=False, xlim=(0, 0.5))

In [ ]:
# df[[c for c in df.columns if 'PC' in c]].describe()

In [ ]:
# for i in range(8):
#     df.plot(x=f'SW[{i}]', y=f'PC[{i}]', kind='scatter', logy=False, xlim=(0, 1))
# plt.show()